This works

In [1]:
import graphix.command
from graphix.random_objects import Circuit, rand_circuit
from graphix.states import BasicStates
from graphix.sim.density_matrix import DensityMatrixBackend
from graphix.sim.statevec import StatevectorBackend, Statevec
from graphix.pauli import Pauli
from graphix.fundamentals import IXYZ
import stim
from graphix.noise_models import DepolarisingNoiseModel

from veriphix.client import Client, Secrets, CircuitUtils
import veriphix.client
import numpy as np
import random

/Users/sabdulsa/Codes/veriphix/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [15]:
nqubits = 2
depth = 20
circuit = rand_circuit(nqubits, depth)
pattern = circuit.transpile().pattern 
pattern.minimize_space()

print(f"Number of nodes in the pattern : {pattern.n_node}")

## Measure output nodes, to have classical output
classical_output = pattern.output_nodes
for onode in classical_output:
    pattern.add(graphix.command.M(node=onode))


Number of nodes in the pattern : 173


Here comes **Veriphix**

In [16]:
secrets = Secrets(r=True, a=True, theta=True)
client = Client(pattern=pattern, secrets=secrets)
test_runs = client.create_test_runs()

# Insert noise here !!
noise = DepolarisingNoiseModel(entanglement_error_prob=0.01)

backend = DensityMatrixBackend()

n_failures = 0
n_iterations = 100
for _ in range(n_iterations) :
    run = random.choice(test_runs)
    trap_outcomes = client.delegate_test_run(run=run, backend=backend, noise_model = noise)
    
    if sum(trap_outcomes) != 0:
        n_failures += 1
    #     print("Failed trap round")
    # else:
    #     print("Trap round passed")
        
print(f"Number of failed rounds: {n_failures}/{n_iterations}")

Number of failed rounds: 55/100


In [14]:
def make_clifford(pattern:graphix.Pattern):
    clifford_pattern = graphix.Pattern(pattern.input_nodes)
    for cmd in pattern:
        clifford_pattern.add(cmd)
    for cmd in clifford_pattern:
        if cmd.kind == graphix.command.CommandKind.M :
            new_angle = random.choice([0, 1/2, 1, 3/2])
            cmd.angle = new_angle
    return clifford_pattern

In [6]:

backend = DensityMatrixBackend()


# Noiseless computation run
client.delegate_pattern(backend=backend)
for onode in classical_output:
    print(client.results[onode])
    

# Noiseless trap run
noiseless_model = DepolarisingNoiseModel(entanglement_error_prob=0)
rd_run = random.choice(runs)
trap_outcomes = client.delegate_test_run(backend=backend, run=rd_run, noise_model=noiseless_model)
print(f"Trap outcome of noiseless trap run: {sum(trap_outcomes)}") # should output 0 (noiseless simulation)

# (ultra) Noisy trap run
ultra_noisy_model = DepolarisingNoiseModel(entanglement_error_prob=1)
rd_run = random.choice(runs)
trap_outcomes = client.delegate_test_run(backend=backend, run=rd_run, noise_model=ultra_noisy_model)
print(f"Trap outcome of noisy trap run: {sum(trap_outcomes)}") # should NOT output 0 (noisy simulation)


0
1
1
1
Trap outcome of noiseless trap run: 0
Trap outcome of noisy trap run: 151
